In [3]:
# === OCMC_1 — # de counterparties únicos en 30 días (por ventana) =============
# Regla: When a Customer transacts with more than {var.Number} Counterparties within 30 days...
# Justificación: distribución del # de counterparties en 30 días (excluye counterparty_id='NA'),
#                escoger el entero que represente anomalías (p.ej., ceil del p95).
import pandas as pd
import numpy as np
from collections import Counter, deque
import math

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# -------- Parámetros editables --------
PATH = "../../data/tx_retail_whale.csv"      # <-- ajusta a tu archivo
WINDOW_DAYS = 30
PCTS = [0.50, 0.75, 0.90, 0.95, 0.97, 0.99]   # percentiles a mostrar

# -------- Carga mínima --------
df = pd.read_csv(PATH, dtype={"customer_id":"string", "counterparty_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]  = pd.to_datetime(df["tx_date_time"], errors="coerce")

# -------- Filtros según regla --------
# Nota: la regla dice "transacts" (no restringimos dirección ni tipo),
#       solo exigimos fecha, customer y counterparty válidos, y excluimos 'NA'.
mask = (
    df["tx_date_time"].notna() &
    df["customer_id"].notna() &
    df["counterparty_id"].notna() &
    (df["counterparty_id"].str.upper().str.strip() != "NA")
)
g = df.loc[mask, ["customer_id", "tx_date_time", "counterparty_id"]].copy()

if g.empty:
    print("No hay transacciones elegibles para OCMC_1 (counterparty_id válidos y fecha).")
else:
    counts = []

    # --- Para cada cliente: sliding window de 30 días con conteo de counterparties únicos ---
    for cid, sub in g.groupby("customer_id", sort=False):
        sub = sub.sort_values("tx_date_time")
        times = sub["tx_date_time"].to_numpy()
        cps   = sub["counterparty_id"].astype(str).to_numpy()

        # Ventana deslizante: mantenemos eventos dentro de [t-30d, t]
        win = deque()                # elementos: (time, cp)
        freq = Counter()             # frecuencia por counterparty en la ventana
        distinct = 0

        left = 0
        for right in range(len(times)):
            t = times[right]
            cp = cps[right]
            # Incluir actual
            win.append((t, cp))
            prev = freq[cp]
            freq[cp] += 1
            if prev == 0:
                distinct += 1

            # Expulsar fuera de la ventana
            cutoff = t - np.timedelta64(WINDOW_DAYS, "D")
            while win and (win[0][0] < cutoff):
                t0, cp0 = win.popleft()
                freq[cp0] -= 1
                if freq[cp0] == 0:
                    distinct -= 1

            # Conteo de counterparties únicos en la ventana que termina en 't'
            counts.append(distinct)

    s = pd.Series(counts, dtype="float")
    if s.empty:
        print("No se pudieron construir ventanas de 30 días.")
    else:
        q = s.quantile(PCTS)
        out = pd.DataFrame({
            "percentil":   [f"p{int(p*100)}" for p in PCTS],
            "Counterparties_30d": [q[p] for p in PCTS],
            "Ceil":        [int(math.ceil(q[p])) for p in PCTS]
        })

        print("=== OCMC_1 — # de counterparties únicos en ventanas de 30 días ===")
        print(f"Puntos (ventanas evaluadas): {len(s):,}")
        display(out)

        # Sugerencia típica (ajústala a tu criterio): usar ceil(p95)
        rec = int(math.ceil(q.get(0.95))) if pd.notna(q.get(0.95, np.nan)) else np.nan
        print(f"\nSugerencia (conservadora): { '{var.Number}' } = ceil(p95) = {rec}")


=== OCMC_1 — # de counterparties únicos en ventanas de 30 días ===
Puntos (ventanas evaluadas): 250


,percentil,Counterparties_30d,Ceil
0,p50,1,1
1,p75,2,2
2,p90,2,2
3,p95,3,3
4,p97,4,4
5,p99,5,5



Sugerencia (conservadora): {var.Number} = ceil(p95) = 3


# Simulación alertas

In [4]:
# === OCMC_1 — Sensibilidad (Actual vs propuestos) ==============================
# LÓGICA EXACTA (según "Rule Logic"):
# unique count(counterparty_id) por {customer_id} en 30 días > [Number]
# AND count de tx por {customer_id & counterparty_id} en 30 días = 1   (es la primera con esa contraparte)
# Excluye counterparty_id = 'NA'
# Unidad = transacciones (las primerizas que cumplen bajo esa condición)

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# --- EDITA AQUÍ ---------------------------------------------------------------
PATH = "../../data/tx_retail_whale.csv"
PARAMS = {
    #"Actual": {"Number": 2},
    "p90":    {"Number": 2},
    "p95":    {"Number": 3},
    "p97":    {"Number": 4},
    "p99":    {"Number": 5},
}
# -----------------------------------------------------------------------------

df = pd.read_csv(PATH, dtype={"customer_id":"string","counterparty_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["counterparty_id"] = df["counterparty_id"].astype(str)

# Filtro base
df = df[(df["tx_date_time"].notna()) & df["customer_id"].notna() & df["counterparty_id"].ne("NA")].copy()
df = df.sort_values(["customer_id","tx_date_time"])

# Precalcular por cliente:
parts = []
for cid, sub in df.groupby("customer_id", sort=False):
    sub = sub.sort_values("tx_date_time").copy()

    # Índice diario continuo en el rango del cliente
    day_idx = pd.date_range(sub["tx_date_time"].min().normalize(),
                            sub["tx_date_time"].max().normalize(), freq="D")

    # Matriz fecha x contraparte con conteos diarios (sin apply; evita MultiIndex raro)
    cp_daily = (
        sub.groupby([pd.Grouper(key="tx_date_time", freq="D"), "counterparty_id"])
           .size()
           .unstack("counterparty_id", fill_value=0)
           .reindex(day_idx, fill_value=0)
    )

    # # de contrapartes únicas activas en ventana 30 días al cierre de cada día
    # (contraparte activa = tuvo ≥1 tx en esa ventana)
    uniq30 = (cp_daily.gt(0).rolling(window=30, min_periods=1).sum()
                         .gt(0).sum(axis=1))  # serie indexada por día

    # Marcar por transacción si es la PRIMERA con esa contraparte en [d-29, d]
    first_flags = []
    for _, row in sub.iterrows():
        d = row["tx_date_time"].normalize()
        mask_30 = (sub["tx_date_time"] >= d - pd.Timedelta(days=29)) & (sub["tx_date_time"] <= d)
        cnt_pair_30 = (sub.loc[mask_30, "counterparty_id"] == row["counterparty_id"]).sum()
        first_flags.append(int(cnt_pair_30 == 1))

    sub["_is_first30"]     = first_flags
    sub["_uniq30_at_day"]  = sub["tx_date_time"].dt.normalize().map(uniq30).astype(float)
    parts.append(sub)

G = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=df.columns.tolist()+["_is_first30","_uniq30_at_day"])

# Tabla de parámetros
order = ["Actual","p50","p75","p90","p95","p97","p99"]
param_tbl = (pd.DataFrame(PARAMS).T
                .loc[[k for k in order if k in PARAMS]]
                .rename_axis("escenario")
                .reset_index())
print("=== OCMC_1 — Parámetros (Number) ===")
display(param_tbl)

# Conteos por escenario
counts = {}
for k, v in PARAMS.items():
    N = v["Number"]
    m = (G["_is_first30"].eq(1) & (G["_uniq30_at_day"] > N))
    counts[k] = int(m.sum())

out = pd.DataFrame([{
    "alertas_actual": counts.get("Actual", 0),
    "alertas_p50":    counts.get("p50", 0),
    "alertas_p75":    counts.get("p75", 0),
    "alertas_p90":    counts.get("p90", 0),
    "alertas_p95":    counts.get("p95", 0),
    "alertas_p97":    counts.get("p97", 0),
    "alertas_p99":    counts.get("p99", 0),
}])

print("=== OCMC_1 — Alertas por escenario (tx primerizas) ===")
display(out)


=== OCMC_1 — Parámetros (Number) ===


,escenario,Number
0,p90,2
1,p95,3
2,p97,4
3,p99,5


=== OCMC_1 — Alertas por escenario (tx primerizas) ===


,alertas_actual,alertas_p50,alertas_p75,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,0,0,0,46,17,8,4
